In [81]:
# !pip install bitsandbytes accelerate

In [11]:
import pandas as pd
import numpy as np

In [12]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

### Подготовка данных

In [14]:
reviews_data = pd.read_csv("C:\\Users\\just_\\Downloads\\archive\\Books_rating.csv", nrows=1000)
reviews_data.head(3)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."


In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
model = model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
def summarize_review(review_text):
    input_text = f"""Your task is to summarize this review: {review_text}"""
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    
    outputs = model.generate(input_ids, 
                             max_length=1024,
                             min_length=8,
                             top_k=150,
                             top_p=0.9,
                             temperature=0.2,
                             do_sample=True)

    return tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True)[0]


In [20]:
reviews_data['summary'] = reviews_data['review/text'].progress_apply(summarize_review)

  2%|█▎                                                                              | 16/1000 [00:50<51:51,  3.16s/it]

KeyboardInterrupt



In [26]:
grouped_reviews = reviews_data.groupby('Id')['summary'].apply(lambda x: list([f'Review {i + 1}: ' + v for i, v in enumerate(x)])).reset_index()
grouped_reviews.head(3)

,Id,summary
0,0007106823,[Review 1: ]
1,0026204207,[Review 1: ]
2,006000486X,"[Review 1: , Review 2: , Review 3: , Review 4:..."


In [ ]:
def summarize_aggregated_reviews(aggregated_reviews):
    input_text = f"Summarize these reviews: {aggregated_reviews}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)

    outputs = model.generate(input_ids,
                             max_length=1024,
                             min_length=8,
                             top_k=150,
                             top_p=0.9,
                             temperature=0.2,
                             do_sample=True)
    return tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True)[0]

In [ ]:
grouped_summaries['final_summary'] = grouped_summaries['aggregated_summaries'].progress_apply(summarize_aggregated_reviews)